# <font color='blue'>Uni-Facef - Pyspark - Parte 5 </font>

Nesse notebook vamos tirar a duplicidade de alguns registros, simulando alterações que foram ocorridas e "appendadas" através de um processo de ingestão no "datalake" 

In [ ]:
# coding: utf-8
import pyspark.sql.functions as sf
from pyspark.sql import Window
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
    .appName('TiraDuplicidade') \
    .getOrCreate()

In [ ]:
cliente_raw = spark.read.parquet('cliente_raw')
cliente_raw.show(10)

#### Cria um "Window" através de uma partição, configurando um grupamento pelo chave

In [ ]:
win_ult_atualizacao = Window.partitionBy("id_cliente") \
    .orderBy(sf.col("dt_ult_atualizacao").desc())

#### Cria uma coluna incremental dentro de cada partição "id_cliente" iniciando pela maoir data

In [ ]:
df_cli_distinct = cliente_raw.dropDuplicates() \
    .withColumn("distinct", sf.row_number().over(win_ult_atualizacao))

df_cli_distinct.show()

In [ ]:
df_cli_trusted = df_cli_distinct.filter("distinct == 1").drop("distinct")

df_cli_trusted.show()

In [ ]:
# Escreve em formato parquet
df_cli_trusted.repartition(2).write \
    .parquet("cliente_trusted", mode="overwrite")

### FIM
###### Documentação: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html